Instalando o openpyxl para permitir leituras de tabelas Excel

In [0]:
pip install openpyxl

Python interpreter will be restarted.
Requirement already satisfied: openpyxl in /databricks/python3/lib/python3.8/site-packages (3.0.10)
Requirement already satisfied: et-xmlfile in /databricks/python3/lib/python3.8/site-packages (from openpyxl) (1.1.0)
Python interpreter will be restarted.

#**Importando bibliotecas**

Foram utilizados as bibliotecas do pyspark, para a leitura e tratamento dos dados na tabela de Datas.
A biblioteca pandas foi utilizada para tratar a tabela de metas da CONSC.
Por fim, as bibliotecas datetime e date foram utilizadas para obter as datas na tabela de metas da CONSC.

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as func
from datetime import date
from datetime import datetime
import pandas as pd

#Definindo para utilizar o datetime Legacy
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

Out[1]: DataFrame[key: string, value: string]

#**Leitura dos arquivos**

Aqui ficam os arquivos lidos para o tratamento.
Utilizou-se em um primeiro momento o ls para verificar o caminho dos documentos a serem buscados.
Os dados dos arquivos calendário e feriados foram os primeiros a serem relacionados, seguidos dos arquivos de dados mensais e semanais das vendas dos carros. Posteriormente, tratamos o arquivo target e por fim tratamos do arquivo metas, relacionando-os com o arquivo de integração, utilizando uma conversão entre os nomes do modelo definido pelo processo De-Para com a tabela criada.

In [0]:
#Busca de arquivos
dbutils.fs.ls("/mnt/blobed/")
#Leitura dos Arquivos

#Datas a relacionar
calendario = spark.read.options(sep=";", header=True, encoding = "UTF-8").csv("/mnt/blobed/modulo_02/calendario/calendario.csv")
feriados = spark.read.options(sep=";", header=True, encoding = "UTF-8").csv("/mnt/blobed/modulo_02/calendario/feriados_nacionais.csv")

#Dados mensais e semanais para unir
dados_mensais = spark.read.options(sep=";", header=True, encoding = "UTF-8").csv("/mnt/blobed/modulo_02/detran/mensal/2022_09_06_detranmensal.csv")
dados_semanais = spark.read.options(sep=";", header=True, encoding = "UTF-8").csv("/mnt/blobed/modulo_02/detran/semanal/2022_09_05_detransemanal.csv")

#Tabela para tratamento
target = spark.read.options(sep=";", header=True, encoding = "UTF-8").csv("/mnt/blobed/modulo_02/target/target.csv")

#Caminho do Arquivo - Mudar para cada novo arquivo meta!
arquivo = 'https://uniedsa.blob.core.windows.net/danielmattos/1.METAS_2022_SET.xlsx'
#Leitura do Excel
metas_pre = pd.read_excel(arquivo, sheet_name='Por Dealer Real')

#Tabelas para integração com a tabela de metas
semanais_integra = spark.read.options(sep=";", header=True, encoding = "UTF-8").csv("/mnt/blobed/modulo_02/detran/semanal/2022_09_13_detransemanal.csv")
integracao = spark.read.options(sep=";", header=True, encoding = "UTF-8").csv("/mnt/blobed/modulo_02/integracao/integracao.csv")

#Leitura da tabela De-Para 
de_para = pd.read_excel('https://uniedsa.blob.core.windows.net/danielmattos/Modelo_De_Para.xlsx', sheet_name = 'Sheet1' \
                        , dtype = {'Modelo Detran' : str, 'Modelo Metas' : str }, usecols = ["Modelo Detran","Modelo Metas"])

* Transformando as datas para a configuração mais comumente aceita : yyyy-mm-dd (Ano-Mês-Dia)

In [0]:
#Calendario - Remover Hora da Coluna Data
#Feriado - Calcular dias e preencher/ Criar Coluna de Dia útil 0/1 e Tipo (Dia útil | fim de semana | nome do feriado)

calendario = calendario.withColumn("Data", func.to_date(func.to_timestamp(col("Data"), "dd/MM/yyyy")))
feriados = feriados.na.drop(how = "any")

feriados = feriados.withColumn("Data", func.to_date(func.to_timestamp(col("Data"), "dd/MM/yyyy")))

#**Tratamento do Calendário**
O principal objetivo dessa tratativa do calendário foi a busca pelos dias úteis, finais de semana e feriados de 2016 a 2024, a fim de averiguar o volume de vendas em relação a distribuição de vendas ao longo dos dias úteis e feriados, permitindo uma maior exatidão na busca por maior engajamento com o cliente, em tais faixas temporais de menor frequência.

In [0]:
#Adicionado ID para obtenção de coluna ID
calendario = calendario.select("*").withColumn("id", monotonically_increasing_id())

#Função de dias da Semana (Segunda-feira = 1)
def diaDaSemanaNumero(colunaNum):
    if colunaNum % 7 == 3:
        return "7"
    elif colunaNum % 7 == 4:
        return "1"
    elif colunaNum % 7 == 5:
        return "2"
    elif colunaNum % 7 == 6:
        return "3"
    elif colunaNum % 7 == 0:
        return "4"
    elif colunaNum % 7 == 1:
        return "5"
    elif colunaNum % 7 == 2:
        return "6"

#Função de averiguação de dia útil ou final de semana em dias que não são feriados
def diaUtil(ColunaSemanaNulls, ColunaTipo):
    if (ColunaSemanaNulls == "6" or ColunaSemanaNulls ==  "7") and ColunaTipo == None:
        return "Fim de Semana"
    elif ColunaTipo == None:
        return "Útil"
    else:
        return ColunaTipo

#Função de adição de coluna 0/1 baseado em dia não útil ou dia útil
def numeroDia(ColunaUtil):    
    if ColunaUtil == "Útil":
        return "1"
    else:
        return "0"

    
diasNum = udf(diaDaSemanaNumero, StringType())
diaTeste = udf(diaUtil, StringType())
diasBinario = udf(numeroDia, StringType())

#Lembrar que 1 é segunda-feira e 7 é domingo
calendario = calendario.withColumn("Número", diasNum('id').cast('int'))

#Left Join do Calendario com Feriados
calendario = calendario.join(feriados,'Data','left')

#Drop do Dia da Semana
calendario = calendario.drop("Dia da Semana")

#Renome da coluna
calendario = calendario.withColumnRenamed("Feriado","Tipo")

#Adição de dias que não são feriados
calendario = calendario.withColumn("Tipo", diaTeste('Número','Tipo').cast('string'))

#Adição do binário
calendario = calendario.withColumn("0-1", diasBinario('Tipo').cast('int'))

#Retirada da coluna id
calendario = calendario.drop("id")

* Demonstração do resultado das junções e tratativas entre as tabelas Calendário e Feriados

In [0]:
#Resultado
display(calendario)

Data,Número,Tipo,0-1
2016-06-30,4,Útil,1
2016-07-01,5,Útil,1
2016-07-02,6,Útil,1
2016-07-03,7,Útil,1
2016-07-04,1,Útil,1
2016-07-05,2,Útil,1
2016-07-06,3,Útil,1
2016-07-07,4,Útil,1
2016-07-08,5,Útil,1
2016-07-09,6,Útil,1


#**Junção das Vendas**
Na trativa abaixo foram realizados a junção das informações de vendas mensais com os dados de vendas da semana posterior a fim de atualizar o banco de dados com um maior número de entradas

In [0]:
#Reformulação da tabela agregando as colunas Ano-Mes-Dia para formar a data
dados_mensais_final = dados_mensais.select(concat_ws('-',col('ano'),col('mes'),col('dia'))\
                         .alias('Data').cast('date'),'marca','modelo','versao','vin')


#Reformulação da tabela agregando as colunas Ano-Mes-Dia para formar a data
dados_semanais_final = dados_semanais.select(concat_ws('-',col('ano'),col('mes'),col('dia'))\
                         .alias('Data').cast('date'),'marca','modelo','versao','vin')

#Join dos dados
dados_final = dados_semanais_final.join(dados_mensais_final,['Data', 'marca', 'modelo', 'versao', 'vin'],'outer')

#Reformulação da tabela agregando as colunas Ano-Mes-Dia para formar a data
semanais_integra_fim = semanais_integra.select(concat_ws('-',col('ano'),col('mes'),col('dia'))\
                         .alias('Data').cast('date'),'marca','modelo','versao','vin')

#Join dos dados da segunda semana de Setembro
vendas_totais = semanais_integra_fim.join(dados_final,['Data', 'marca', 'modelo', 'versao', 'vin'],'outer')

#Ordenação por datas
vendas_totais = vendas_totais.orderBy('Data')

display(vendas_totais)

Data,marca,modelo,versao,vin
2022-01-03,TOYOTA,HILUX,HILUX DOBLE CABINA 2.4,8AJCB3DD3N3908324
2022-01-03,FORD,NEW RANGER,RANGER DCAB XLT 4X4 3.2,8AFAR23W1NJ261041
2022-01-03,TOYOTA,COROLLA CROSS,COROLLA CROSS CVT HEV 1.8 AUT,9BRKZAAG9N0619955
2022-01-03,HYUNDAI,GRAND I-10 AI3 HB 1.2,GRAND I-10 AI3 HB 1.2,MALB251CANM236514
2022-01-03,FORD,NEW RANGER,RANGER DCAB XLT 4X4 3.2,8AFAR23W4NJ261163
2022-01-03,CHEVROLET,TAHOE,TAHOE 4X4 5.3 AT,1GNSK8KD7MR440677
2022-01-03,HYUNDAI,HD 35,HD 35 LWB 2.5,KMFVA17RPNC356649
2022-01-03,TOYOTA,HILUX,HILUX DOBLE CABINA 2.4,8AJJB3DD3N4234970
2022-01-03,TOYOTA,FORTUNER,FORTUNER 4X4 5P 2.8 AUT,8AJDA3FS1N0504152
2022-01-03,TOYOTA,HILUX,HILUX PICK UP 4X4 2P 2.4,8AJDB8CB8N5511794


#**Indicadores de Vendas Mensais**
Os dados das tabelas de vendas mensais foram utilizados para verificar a quantidade de vendas, a marca mais vendida e a distribuição mensal de carros da marca FIAT vendidos

Indicador de quantidade de vendas mensais

In [0]:
#  Qual a soma de vendas por mês? 

#Agrupamento dos meses para quantificar vendais mensais
dados_mes = vendas_totais.groupBy(month(vendas_totais.Data).alias('Mês')).count()

#Ordenação dos dados
dados_mes = dados_mes.orderBy('Mês')

#Renomeação da coluna count
dados_mes = dados_mes.withColumnRenamed('count','Quantidade de Vendas')

display(dados_mes)

Mês,Quantidade de Vendas
1,5453
2,4604
3,5845
4,5740
5,5496
6,6272
7,5885
8,5358
9,1224


* Indicador da marca mais vendida.

In [0]:
#Qual a marca que mais vende?

#Agrupamento das marcas
marcas_vendidas = vendas_totais.groupBy(('marca')).count()

#Ordenação decrescente
marcas_vendidas = marcas_vendidas.orderBy(col('count').desc())

#Renomeação da coluna count
marcas_vendidas = marcas_vendidas.withColumnRenamed('count','Quantidade de Vendas')
#Renomeação da coluna marca
marcas_vendidas = marcas_vendidas.withColumnRenamed('marca','Marcas mais vendidas')

display(marcas_vendidas.head(1))

Marcas mais vendidas,Quantidade de Vendas
CHEVROLET,16992


* Indicador da distribuição de comercialização de carros da marca FIAT ao longo do ano

In [0]:
#Qual a distribuição de vendas da Fiat ao longo do ano?

#Agrupamento dos fiats
fiats = vendas_totais.where("marca = 'FIAT'").groupby(month(vendas_totais.Data).alias("Mês")).count()

#Renomeação da coluna count
fiats = fiats.withColumnRenamed('count','Quantidade de Vendas')

#Ordenação por mês
fiats = fiats.orderBy("Mês")

display(fiats)

Mês,Quantidade de Vendas
1,8
2,27
3,8
4,36
5,150
6,139
7,54
8,39
9,25


#Metas de vendas semestrais
Após a união das vendas ao longo do semestre foi realizado uma verificação para averiguar o cumprimento da meta semestral por modelo de carro vendido. Vale ressaltar que alguns modelos ultrapassaram suas metas.

In [0]:
#Filtro de meses nos dados de vendas
dados_semestre = vendas_totais.filter(month(vendas_totais.Data) <= 6)

#Contagem da junção das marcas e modelos
dados_semestre = dados_semestre.groupby('marca','modelo').count()

#Renomeação da Coluna Count
dados_semestre = dados_semestre.withColumnRenamed('count','Quantidade Vendida')

#Substituindo o tipo string para int da coluna Target Semestre 1
target = target.withColumn("Target Semestre 1", col("Target Semestre 1").cast("int"))

#União do banco target e vendas semestrais
target_semestre = target.join(dados_semestre, ['modelo','marca'], 'outer')

#Filtro de modelos de carro cuja meta foi atingida
target_metas = target_semestre.filter(col("Target Semestre 1") <= col("Quantidade Vendida"))
display(target_metas)

Modelo,Marca,Target Semestre 1,Quantidade Vendida
LANDTREK,PEUGEOT,207,207
TRAVERSE,CHEVROLET,97,97
FORTUNER,TOYOTA,5,98
AMAROK,VOLKSWAGEN,173,173
520,BMW,3,3
STARIA FG,HYUNDAI,9,9
NEW ESCAPE,FORD,43,43
SANTA FE TM FL,HYUNDAI,16,16
F-250,FORD,2,2
X1,BMW,1,10


#Relação de vendas por modelo
Nessa tratativa, o foco dá-se nos modelos vendidos em cada loja, permitindo a verificação de venda de um modelo específico com maior facilidade.

In [0]:
#Drop das linhas de espaçamento
metas_final = metas_pre.dropna(how = "all")

#Obter apenas o nome do arquivo
arquivo = arquivo.split('/')
nomeArquivo = arquivo[len(arquivo) - 1]

#Obter Data da Planilha
data = pd.to_datetime(metas_pre.columns[2]).strftime('%Y-%m-%d')

#Retirando o nome das colunas
nomeColunas = list(metas_pre.iloc[3].dropna())
i = nomeColunas.index('TOTAL')
del nomeColunas[i : len(nomeColunas)]


#Inserindo Nome das Colunas CodSAP e Modelo
nomeColunas.insert(0,"CodSAP")
nomeColunas.insert(1,"Modelo")

* Retirando colunas vazias e renomeando colunas para facilitar a análise.

In [0]:
#Dropando colunas
metas_final = metas_final.drop([0]).drop("Unnamed: 0",axis = 1)

#Dropando as linhas de total (Retirando por Código)
metas_final = metas_final.dropna(subset = ["CONSC"])

#Renomeando colunas
metas_final = metas_final.drop([2]).rename(columns= {"CONSC":"CodSAP",data: "Modelo" })

#Tipificando corretamente a tabela CodSAP para String
metas_final= metas_final.astype({'CodSAP' : 'str'})

#Dropando as colunas de total por carro
metas_final = metas_final.dropna(axis = 1)

#Renomeação das colunas
metas_final.columns = nomeColunas

* Despivotando as colunas dos Dealers para focar a análise no volume de vendas por marca

In [0]:
#Melt das colunas
metas_final = pd.melt(metas_final, id_vars = ["CodSAP","Modelo"])

#Renomeação da coluna de Dealers e Quantidade
metas_final = metas_final.rename(columns = {"value":"Quant","variable":"Dealer"})

#Ordenação por Modelo para reorganização
metas_final = metas_final.sort_values(["CodSAP","Dealer"])

* Adição da dimensão de temporalidade da tratativa, junto ao nome do arquivo tratado, caso sejam realizados múltiplas tratativas em um mesmo pipeline

In [0]:
#Reseta e organiza índice
metas_final = metas_final.reset_index()
metas_final = metas_final.drop("index", axis = 1)
metas_final.index += 1

#Inserindo Data na coluna
metas_final.insert(0,"Data", data)

#Inserindo filename na coluna
metas_final.insert(5,"filename", nomeArquivo)

#Inserindo timestamp na coluna para temporização de tratamento
metas_final.insert(6,"timestamp", pd.Timestamp.now())

In [0]:
#Display da tabela de metas
display(metas_final)

Data,CodSAP,Modelo,Dealer,Quant,filename,timestamp
2022-09-11,2065,PARTNER 3.1,DL-A,19,1.METAS_2022_SET.xlsx,2022-09-23T22:18:50.821+0000
2022-09-11,2065,PARTNER 3.1,DL-B,16,1.METAS_2022_SET.xlsx,2022-09-23T22:18:50.821+0000
2022-09-11,2065,PARTNER 3.1,DL-C,17,1.METAS_2022_SET.xlsx,2022-09-23T22:18:50.821+0000
2022-09-11,2065,PARTNER 3.1,DL-D,4,1.METAS_2022_SET.xlsx,2022-09-23T22:18:50.821+0000
2022-09-11,2065,PARTNER 3.1,DL-E,3,1.METAS_2022_SET.xlsx,2022-09-23T22:18:50.821+0000
2022-09-11,2065,PARTNER 3.1,DL-F,9,1.METAS_2022_SET.xlsx,2022-09-23T22:18:50.821+0000
2022-09-11,2065,PARTNER 3.1,DL-G,5,1.METAS_2022_SET.xlsx,2022-09-23T22:18:50.821+0000
2022-09-11,2065,PARTNER 3.1,DL-H,10,1.METAS_2022_SET.xlsx,2022-09-23T22:18:50.821+0000
2022-09-11,2065,PARTNER 3.1,DL-I,5,1.METAS_2022_SET.xlsx,2022-09-23T22:18:50.821+0000
2022-09-11,2065,PARTNER 3.1,DL-J,1,1.METAS_2022_SET.xlsx,2022-09-23T22:18:50.821+0000


#Integração dos dados
Nessa próxima célula, os dados de venda totais foram unidos aos valores da integração para obtenção da venda de cada modelo pelos dealers.

Em um segundo momento, as metas mensais são comparadas com os volumes de venda do último mês (Semanas 5 e 13 de Setembro), a fim de verificar o cumprimento das metas.

In [0]:
#Junção entre a tabela de vendas totais e a tabela de integração
vendas_totais = vendas_totais.join(integracao,'vin','left')

In [0]:
#Demonstração da união entre as outras tabelas de vendas
display(vendas_totais)

#Display da tabela de metas
display(metas_final)

vin,Data,marca,modelo,versao,Dealer
9BG148MK0NC431959,2022-01-03,CHEVROLET,COLORADO,COLORADO DCAB 4X4 2.8 AT,DL-I
LZWADAGA2NG005841,2022-01-03,CHEVROLET,GROOVE,GROOVE LTZ 1.5,DL-E
LSGHD52H2ND014025,2022-01-03,CHEVROLET,SAIL,SAIL LS 1.5,DL-N
1GNSK9KDXMR463585,2022-01-03,CHEVROLET,SUBURBAN,SUBURBAN 4X4 5.3L AUT,DL-J
3FMCR9B61MRB11358,2022-01-03,FORD,BRONCO SPORT,BRONCO SPORT 1.5,DL-N
1FTFW1E10MFC13923,2022-01-03,FORD,F-150,F150 LARIAT LUXURY 4X4 3.0,DL-I
8AFAR23W1NJ261041,2022-01-03,FORD,NEW RANGER,RANGER DCAB XLT 4X4 3.2,DL-C
8AFAR23W4NJ261163,2022-01-03,FORD,NEW RANGER,RANGER DCAB XLT 4X4 3.2,DL-B
MALC741BAMM295803,2022-01-03,HYUNDAI,ACCENT HCI,ACCENT HCI 1.4,DL-K
MALC741BAMM295811,2022-01-03,HYUNDAI,ACCENT HCI,ACCENT HCI 1.4,DL-K


Data,CodSAP,Modelo,Dealer,Quant,filename,timestamp
2022-09-11,2065,PARTNER 3.1,DL-A,19,1.METAS_2022_SET.xlsx,2022-09-23T22:18:50.821+0000
2022-09-11,2065,PARTNER 3.1,DL-B,16,1.METAS_2022_SET.xlsx,2022-09-23T22:18:50.821+0000
2022-09-11,2065,PARTNER 3.1,DL-C,17,1.METAS_2022_SET.xlsx,2022-09-23T22:18:50.821+0000
2022-09-11,2065,PARTNER 3.1,DL-D,4,1.METAS_2022_SET.xlsx,2022-09-23T22:18:50.821+0000
2022-09-11,2065,PARTNER 3.1,DL-E,3,1.METAS_2022_SET.xlsx,2022-09-23T22:18:50.821+0000
2022-09-11,2065,PARTNER 3.1,DL-F,9,1.METAS_2022_SET.xlsx,2022-09-23T22:18:50.821+0000
2022-09-11,2065,PARTNER 3.1,DL-G,5,1.METAS_2022_SET.xlsx,2022-09-23T22:18:50.821+0000
2022-09-11,2065,PARTNER 3.1,DL-H,10,1.METAS_2022_SET.xlsx,2022-09-23T22:18:50.821+0000
2022-09-11,2065,PARTNER 3.1,DL-I,5,1.METAS_2022_SET.xlsx,2022-09-23T22:18:50.821+0000
2022-09-11,2065,PARTNER 3.1,DL-J,1,1.METAS_2022_SET.xlsx,2022-09-23T22:18:50.821+0000


* Criação do modelo De-Para através da obtenção de valores únicos colocados em uma tabela no excel e comparados manualmente, nota-se que apenas 3 modelos possuem o mesmo nome.

In [0]:
#Criação do modelo De-Para
modelos_detran =  sorted(vendas_totais.select(func.collect_set('modelo').alias('Modelo Detran')).first()['Modelo Detran'])
modelos_metas = sorted(metas_final.Modelo.unique())

#Criação e demonstração dos nomes iguais
nomes_iguais = set(modelos_detran).intersection(modelos_metas)
print (nomes_iguais)

#Demonstração dos nomes dos modelos
print('\nModelos Detran')
print(modelos_detran)
print('\nModelos Metas')
print(modelos_metas)

{'DUCATO', 'N400 MAX', 'SANTA FE TM'}

Modelos Detran
['118', '2008', '208', '218I', '220', '235', '3008', '301', '308', '320', '330', '420', '430I', '4RUNNER', '5008', '520', '530', '595', 'ACCENT', 'ACCENT HCI', 'AMAROK', 'ARGO', 'ARKANA', 'ATOS AH2', 'BLAZER', 'BOXER', 'BRONCO SPORT', 'C', 'C-HR', 'CAMARO', 'CAPTIVA', 'CASA RODANTE', 'CINQUECENTO', 'CLIO V', 'COLORADO', 'CONSTELLATION', 'COROLLA', 'COROLLA CROSS', 'COUNTY', 'CRETA', 'CRETA GRAND SU2I', 'CRONOS', 'D-MAX', 'DELIVERY', 'DOKKER', 'DUCATO', 'DUSTER', 'ECOSPORT', 'EDGE', 'ELANTRA CN7', 'EX 9', 'EXPEDITION', 'EXPERT', 'EXPLORER', 'EXPRESS', 'F 350', 'F 450', 'F-150', 'F-250', 'FIORINO', 'FORTUNER', 'FTR', 'FVR', 'GOL', 'GRAND I-10', 'GRAND I-10 AI3 HB 1.2', 'GRAND I-10 Ai3 SEDAN ', 'GRAND I-10 SEDAN', 'GROOVE', 'H-1', 'HD 35', 'HIACE', 'HILUX', 'I20 BI3', 'I20 IB', 'I3', 'IONIQ', 'KANGOO', 'KOLEOS', 'KONA', 'KWID', 'LAND CRUISER', 'LANDTREK', 'MAVERICK', 'MIGHTY EX6', 'MIGHTY EX8', 'MOBI', 'MUSTANG', 'N400 MAX', 'NEW CRETA', 'NEW ESCAPE', 'NEW RANGER', 'NIVUS', 'NKR', 'NPR', 'NQR', 'NUEVO VIRTUS', 'ONIX', 'OPTRA', 'OROCH', 'PALISADE LX2', 'PARTNER', 'POLO', 'PORTER', 'PRIUS', 'PULSE', 'RAIZE', 'RAV4', 'RIFTER', 'RUSH', 'SAIL', 'SANTA FE TM', 'SANTA FE TM FL', 'SAVEIRO', 'SILVERADO', 'SIN MODELO', 'SOLATI', 'SPARK GT', 'SPIN', 'STARIA FG', 'STARIA MB', 'SUBURBAN', 'SYMBOL', 'T-CROSS', 'TACOMA', 'TAHOE', 'TAOS', 'TERRITORY', 'TIGUAN', 'TOUAREG', 'TRACKER', 'TRAILBLAZER', 'TRANSIT VAN', 'TRAVERSE', 'TUCSON NX4', 'TUCSON TL', 'VELOSTER', 'VENUE', 'VERNA', 'VOYAGE', 'X1', 'X2 ', 'X3', 'X4', 'X5', 'X6', 'X7', 'YARIS']

Modelos Metas
['235 1L COMFORT ', '301 TM', '420 1.8L COMFORT ', '500 1.5 COMFORT', '500 1.5 LUXURY', '5008 CS', '520 1.8L COMFORT 2R', 'ARGO 4.3T', 'C-HR AC', 'DUCATO', 'ELANTRA CN7 TM', 'F-150 CS', 'MOBI SERIE C', 'N400 MAX', 'NEW ESCAPE XL EV', 'ONIX 1.3 SERIE ', 'PARTNER 3.1', 'PULSE XXL', 'SANTA FE TM', 'SANTA FE TM FL 1.0', 'SILVERADO 1.3', 'SPARK GT.4', 'STARIA FG 5.SC', 'TACOMA 1.3 TM', 'TRAILBLAZER CST']

#Atingimento de metas nos primeiros nove meses e metas mensais em Setembro
Nas próximas celulas serão analisados o atingimento das metas de vendas por modelo no mês de Setembro, através da comparação entre a tabela Metas de Setembro e as vendas na primeira quinzena de Setembro. Em um segundo momento será analisado o atingimento da meta semestral por Dealer, relacionado aos modelos de carros desejados.

* Criação do modelo De-Para a fim de harmonizar os dados de modelos entre as tabelas Metas de Setembro e Vendas

In [0]:
#Renomeando coluna para o join
de_para = de_para.rename(columns = {'Modelo Metas' : 'Modelo'})

#Retirando o espaço dado nos valores da tabela
de_para['Modelo'] = de_para['Modelo'].str.lstrip()
de_para['Modelo Detran'] = de_para['Modelo Detran'].str.lstrip()

#Realizando o join
metas_para = metas_final.merge(de_para, how = 'inner')

#Drop da tabela de modelo antiga e renome para a união
metas_para = metas_para.drop(['Modelo'], axis = 1).rename(columns = {'Modelo Detran' : 'modelo'})

#Tipificação correta da coluna modelo para String
metas_para = metas_para.astype({'modelo' : 'str'})

* União dos dados da última semana (Tabela Semana 13/09) com a tabela Semana 5/09

In [0]:
#União dos dados de Setembro (1/09-10/09)
dados_setembro = dados_semanais_final.union(semanais_integra_fim)
 
#Conversão da tabela metas_para de Pandas para Pyspark
metas_para = spark.createDataFrame(metas_para)

* Agrupamento da quantidade de vendas por modelos e verificação do cumprimento da meta estipulada. Nota-se que nenhum modelo alcançou o número esperado de vendas no mês

In [0]:
#Agrupamento da tabela metas_para por modelos
metas_agg = metas_para.groupBy(['modelo']).sum().withColumnRenamed('sum(Quant)','Meta')

#Agrupamento da tabela dados_setembro por modelos
setembro_agg = dados_setembro.groupBy(['modelo']).count().withColumnRenamed('count','Vendas')

#Junção do banco metas e dados de Setembro para verificação do atingimento da meta
metas_setembro = setembro_agg.join(metas_agg,'modelo' ,'inner')

#Verificação de atingimento das metas de Setembro
entrega_metas = metas_setembro.filter(col('Vendas') >= col('Meta'))
display(entrega_metas)

modelo,Vendas,Meta
PULSE,23,119
SANTA FE TM,5,125
5008,4,118
C-HR,2,104
N400 MAX,48,148
ONIX,37,133
ARGO,4,136
301,7,139
SILVERADO,1,152
STARIA FG,2,128


modelo,Vendas
T-CROSS,3
GRAND I-10 AI3 HB 1.2,25
SAVEIRO,9
308,21
GRAND I-10 SEDAN,5
SAIL,61
DUSTER,7
CAPTIVA,24
PULSE,23
NEW CRETA,37


modelo,Vendas,Meta


* Ordenação e verificação da igualdade entre o nome dos modelos na tabela de Venda e o nome dos modelos na tabela de Metas Semestrais

In [0]:
#Criação do modelo De-Para (Vendas Totais para Target)
modelos_detran =  sorted(vendas_totais.select(func.collect_set('modelo').alias('Modelo Detran')).first()['Modelo Detran'])
modelos_target =  sorted(target.select(func.collect_set('Modelo').alias('Modelo Target')).first()['Modelo Target'])

#Verificação de igualdade de nomes
nomes_iguais = set(modelos_detran).intersection(modelos_target)
print ('Modelos Detran \n', modelos_detran)
print('\nModelos Target \n', modelos_target)

print(len(modelos_target))
print(len(nomes_iguais))
#Todos são iguais! :D

Modelos Detran 
 ['118', '2008', '208', '218I', '220', '235', '3008', '301', '308', '320', '330', '420', '430I', '4RUNNER', '5008', '520', '530', '595', 'ACCENT', 'ACCENT HCI', 'AMAROK', 'ARGO', 'ARKANA', 'ATOS AH2', 'BLAZER', 'BOXER', 'BRONCO SPORT', 'C', 'C-HR', 'CAMARO', 'CAPTIVA', 'CASA RODANTE', 'CINQUECENTO', 'CLIO V', 'COLORADO', 'CONSTELLATION', 'COROLLA', 'COROLLA CROSS', 'COUNTY', 'CRETA', 'CRETA GRAND SU2I', 'CRONOS', 'D-MAX', 'DELIVERY', 'DOKKER', 'DUCATO', 'DUSTER', 'ECOSPORT', 'EDGE', 'ELANTRA CN7', 'EX 9', 'EXPEDITION', 'EXPERT', 'EXPLORER', 'EXPRESS', 'F 350', 'F 450', 'F-150', 'F-250', 'FIORINO', 'FORTUNER', 'FTR', 'FVR', 'GOL', 'GRAND I-10', 'GRAND I-10 AI3 HB 1.2', 'GRAND I-10 Ai3 SEDAN ', 'GRAND I-10 SEDAN', 'GROOVE', 'H-1', 'HD 35', 'HIACE', 'HILUX', 'I20 BI3', 'I20 IB', 'I3', 'IONIQ', 'KANGOO', 'KOLEOS', 'KONA', 'KWID', 'LAND CRUISER', 'LANDTREK', 'MAVERICK', 'MIGHTY EX6', 'MIGHTY EX8', 'MOBI', 'MUSTANG', 'N400 MAX', 'NEW CRETA', 'NEW ESCAPE', 'NEW RANGER', 'NIVUS', 'NKR', 'NPR', 'NQR', 'NUEVO VIRTUS', 'ONIX', 'OPTRA', 'OROCH', 'PALISADE LX2', 'PARTNER', 'POLO', 'PORTER', 'PRIUS', 'PULSE', 'RAIZE', 'RAV4', 'RIFTER', 'RUSH', 'SAIL', 'SANTA FE TM', 'SANTA FE TM FL', 'SAVEIRO', 'SILVERADO', 'SIN MODELO', 'SOLATI', 'SPARK GT', 'SPIN', 'STARIA FG', 'STARIA MB', 'SUBURBAN', 'SYMBOL', 'T-CROSS', 'TACOMA', 'TAHOE', 'TAOS', 'TERRITORY', 'TIGUAN', 'TOUAREG', 'TRACKER', 'TRAILBLAZER', 'TRANSIT VAN', 'TRAVERSE', 'TUCSON NX4', 'TUCSON TL', 'VELOSTER', 'VENUE', 'VERNA', 'VOYAGE', 'X1', 'X2 ', 'X3', 'X4', 'X5', 'X6', 'X7', 'YARIS']

Modelos Target 
 ['118', '2008', '208', '218I', '220', '235', '3008', '301', '308', '320', '330', '420', '430I', '4RUNNER', '5008', '520', '530', 'ACCENT', 'ACCENT HCI', 'AMAROK', 'ARGO', 'ARKANA', 'ATOS AH2', 'BLAZER', 'BOXER', 'BRONCO SPORT', 'C-HR', 'CAMARO', 'CAPTIVA', 'CASA RODANTE', 'CINQUECENTO', 'CLIO V', 'COLORADO', 'COROLLA', 'COROLLA CROSS', 'CRETA', 'CRETA GRAND SU2I', 'CRONOS', 'D-MAX', 'DOKKER', 'DUCATO', 'DUSTER', 'ECOSPORT', 'EDGE', 'ELANTRA CN7', 'EXPEDITION', 'EXPERT', 'EXPLORER', 'EXPRESS', 'F 350', 'F 450', 'F-150', 'F-250', 'FIORINO', 'FORTUNER', 'GOL', 'GRAND I-10', 'GRAND I-10 AI3 HB 1.2', 'GRAND I-10 Ai3 SEDAN ', 'GRAND I-10 SEDAN', 'GROOVE', 'H-1', 'HD 35', 'HIACE', 'HILUX', 'I20 BI3', 'I20 IB', 'I3', 'IONIQ', 'KANGOO', 'KOLEOS', 'KWID', 'LAND CRUISER', 'LANDTREK', 'MAVERICK', 'MOBI', 'MUSTANG', 'N400 MAX', 'NEW CRETA', 'NEW ESCAPE', 'NEW RANGER', 'NIVUS', 'NUEVO VIRTUS', 'ONIX', 'OPTRA', 'OROCH', 'PALISADE LX2', 'PARTNER', 'POLO', 'PORTER', 'PRIUS', 'PULSE', 'RAIZE', 'RAV4', 'RIFTER', 'RUSH', 'SAIL', 'SANTA FE TM', 'SANTA FE TM FL', 'SAVEIRO', 'SILVERADO', 'SIN MODELO', 'SOLATI', 'SPARK GT', 'SPIN', 'STARIA FG', 'STARIA MB', 'SUBURBAN', 'SYMBOL', 'T-CROSS', 'TACOMA', 'TAHOE', 'TAOS', 'TERRITORY', 'TIGUAN', 'TRACKER', 'TRAILBLAZER', 'TRANSIT VAN', 'TRAVERSE', 'TUCSON NX4', 'TUCSON TL', 'VELOSTER', 'VENUE', 'VERNA', 'VOYAGE', 'X1', 'X2 ', 'X3', 'X4', 'X5', 'X6', 'YARIS']
132
132

* Agrupamento e filtragem para obtenção dos Dealers cujos modelos atingiram a meta de volume de vendas, a fim de demonstrar o cumprimento de tal meta

In [0]:
#Renomeação para o join
target = target.withColumnRenamed('Modelo','modelo')

#Agrupamento dos modelos vendidos por dealer
totais_entrega = vendas_totais.groupBy(['modelo','Dealer']).count().withColumnRenamed('count','Quantidade de Vendas')

#Junção para averiguar atingimento da meta semestral
juncao_semestral = target.join(totais_entrega,'modelo','inner')

#Verificação de atingimento da meta semestral
analise_semestral = juncao_semestral.filter(col('Target Semestre 1') <= col('Quantidade de Vendas')).orderBy('Dealer')

#Reorganização das colunas
analise_semestral = analise_semestral.select('Marca','modelo','Dealer','Target Semestre 1','Quantidade de Vendas')

#Demonstração do resultado
display(analise_semestral)

Marca,modelo,Dealer,Target Semestre 1,Quantidade de Vendas
FIAT,FIORINO,DL-A,1,1
TOYOTA,FORTUNER,DL-A,5,8
VOLKSWAGEN,NUEVO VIRTUS,DL-A,1,3
PEUGEOT,308,DL-A,11,12
HYUNDAI,TUCSON NX4,DL-A,5,41
BMW,X1,DL-A,1,1
FORD,F 450,DL-A,1,1
FIAT,MOBI,DL-B,1,1
TOYOTA,FORTUNER,DL-B,5,8
VOLKSWAGEN,NUEVO VIRTUS,DL-B,1,2


* Junção entre a tabela calendário e a tabela de vendas totais e consultas baseadas em datas

In [0]:
#Junção das tabelas calendario e vendas totais
vendas_calendario = vendas_totais.join(calendario,'Data','inner')
display(vendas_calendario)

#Agrupamento por dia da semana
dias_semana_vendas = vendas_calendario.groupby('Número').count() 
dias_semana_vendas = dias_semana_vendas.orderBy('Número').withColumnRenamed('count','Quantidade de Vendas')
display(dias_semana_vendas)

#Agrupamento por dealer por dia da semana
dealer_semana_vendas = vendas_calendario.groupby('Dealer','Número').count() 
dealer_semana_vendas = dealer_semana_vendas.orderBy('Número').withColumnRenamed('count','Quantidade de Vendas')
display(dealer_semana_vendas)

Data,vin,marca,modelo,versao,Dealer,Número,Tipo,0-1
2022-01-03,9BG148MK0NC431959,CHEVROLET,COLORADO,COLORADO DCAB 4X4 2.8 AT,DL-I,1,Útil,1
2022-01-03,LZWADAGA2NG005841,CHEVROLET,GROOVE,GROOVE LTZ 1.5,DL-E,1,Útil,1
2022-01-03,LSGHD52H2ND014025,CHEVROLET,SAIL,SAIL LS 1.5,DL-N,1,Útil,1
2022-01-03,1GNSK9KDXMR463585,CHEVROLET,SUBURBAN,SUBURBAN 4X4 5.3L AUT,DL-J,1,Útil,1
2022-01-03,3FMCR9B61MRB11358,FORD,BRONCO SPORT,BRONCO SPORT 1.5,DL-N,1,Útil,1
2022-01-03,1FTFW1E10MFC13923,FORD,F-150,F150 LARIAT LUXURY 4X4 3.0,DL-I,1,Útil,1
2022-01-03,8AFAR23W1NJ261041,FORD,NEW RANGER,RANGER DCAB XLT 4X4 3.2,DL-C,1,Útil,1
2022-01-03,8AFAR23W4NJ261163,FORD,NEW RANGER,RANGER DCAB XLT 4X4 3.2,DL-B,1,Útil,1
2022-01-03,MALC741BAMM295803,HYUNDAI,ACCENT HCI,ACCENT HCI 1.4,DL-K,1,Útil,1
2022-01-03,MALC741BAMM295811,HYUNDAI,ACCENT HCI,ACCENT HCI 1.4,DL-K,1,Útil,1


Número,Quantidade de Vendas
1,8810
2,8195
3,9989
4,9055
5,8069
6,1737
7,22


Dealer,Número,Quantidade de Vendas
DL-A,1,668
DL-F,1,633
DL-L,1,664
DL-J,1,607
DL-M,1,609
DL-C,1,663
DL-H,1,655
DL-D,1,634
DL-K,1,568
DL-I,1,640
